In [52]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
import random
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [53]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


In [54]:
list_of_action = {
    0: (0,0),
    1: (0,1),
    2: (0,2),
    3: (1,0),
    4: (1,1),
    5: (1,2),
    6: (2,0),
    7: (2,1),
    8: (2,2)
}

rev_action = {
    (0,0) :0,
    (0,1) :1,
    (0,2) :2,
    (1,0) :3,
    (1,1) :4,
    (1,2) :5,
    (2,0) :6,
    (2,1) :7,
    (2,2) :8
}

In [55]:
class DQN(nn.Module):

    def __init__(self, in_features, outputs, action_list):
        super(DQN, self).__init__()
        
        self.action_list = action_list
     
        self.head1 = nn.Linear(in_features, 128)
        self.head2 = nn.Linear(128,outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        #print('input x', x.shape, type(x))
        x = F.relu(self.head1(x.view(x.size(0), -1)))
        x = torch.sigmoid(self.head2(x))
        #print('output x',x.shape)
        return x
    
    def act(self, obs):
        obs_tensor = torch.as_tensor(obs, dtype=torch.float32)
        #mask_possible_action = torch.ne(obs_tensor,0)*-1
        #print('mask_possible_action',mask_possible_action)
        
        q_values = self.forward(obs_tensor.unsqueeze(0))
        #possible_q_values = mask_possible_action*q_values
        #print('possible_q_values',possible_q_values)
        
        
        max_q_idx = torch.argmax(q_values, dim=1)[0]
        max_q_idx_item = max_q_idx.detach().item()
        #print('max_q_idx_item', max_q_idx_item, type(max_q_idx_item))
        #print('action list', self.action_list)
        #print('rev_action',rev_action)
        move = self.action_list[max_q_idx_item]
        
        action_idx = rev_action[move]
        
        return action_idx
    
        

In [56]:
BATCH_SIZE = 32
BUFFER_SIZE = 50
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE_FREQ = 10

number_of_actions = 9
grid_size = 9


env = TictactoeEnv()
replay_buffer = deque(maxlen=BUFFER_SIZE)
rew_buffer = deque([0.0], maxlen=10)

episode_reward = 0.0

#initialization network
policy_net = DQN(grid_size, number_of_actions,list_of_action).to("cpu")
target_net = DQN(grid_size, number_of_actions,list_of_action).to("cpu")
target_net.load_state_dict(policy_net.state_dict())
#target_net.eval()





<All keys matched successfully>

In [59]:
Turns = np.array(['X','O'])
MIN_REPLAY_SIZE = 40

#Initialize the Replay Buffer
replay_buffer = deque(maxlen=BUFFER_SIZE)

game = 0
episode_reward = 0.0

while(len(replay_buffer) < MIN_REPLAY_SIZE):
    game += 1
    print('game', game, 'buffer', len(replay_buffer))
    env.reset()
    #env.render()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt = OptimalPlayer(epsilon=0.7, player='O')
    player_rnd = OptimalPlayer(epsilon=1., player='X')
    
    for j in range(5):
        #print('step', j)
       
        #random agent play
        obs = env.observe()[0].flatten()
        move = player_rnd.act(grid)
        grid, end, winner = env.step(move, print_grid=False)
        action = rev_action[move]
        
        if not end:
            #optimal player play
            move = player_opt.act(grid)
            grid, end, winner = env.step(move, print_grid=False)
        
        new_obs = env.observe()[0].flatten()
       
    
        #transition set-off
        rew = env.reward(player=Turns[1])
        transition = (obs, action, rew, end, new_obs)
        replay_buffer.append(transition)
           

        if end:
            #print('-------------------------------------------')
            #print('Game end, winner is player ' + str(winner))
            #print('Optimal player = ' +  Turns[0])
            #print('Random player = ' +  Turns[1])
            #env.render()
            env.reset()
            break




#for _ in range(MIN_REPLAY_SIZE):
    
   

game 1 buffer 0
game 2 buffer 5
game 3 buffer 9
game 4 buffer 13
game 5 buffer 16
game 6 buffer 21
game 7 buffer 26
game 8 buffer 30
game 9 buffer 34
game 10 buffer 39


In [60]:
print(len(replay_buffer))

43


In [61]:
print(type(replay_buffer))

<class 'collections.deque'>


In [62]:
print(replay_buffer[1])

(array([ 1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.]), 1, 0, False, array([ 1.,  1., -1.,  0.,  0.,  0., -1.,  0.,  0.]))


In [63]:
#Main Training Loop

env.reset()
env.render()
player_opt = OptimalPlayer(epsilon=0.7, player='O')

optimizer = torch.optim.Adam(policy_net.parameters(), lr=5e-4)
rew_buffer = deque([0.0], maxlen=50)
bad_pred_buffer = deque([0.0], maxlen=50)

game = 0
old_game = 0
step = 0
bad_prediction = False

episode_reward = 0.0
episode_bad_predict = 0.0
while(game < 500):
    step +=1
    
   
    #random agent play
    
    move = policy_net.act(obs)
    obs = env.observe()[0]
    available_action = find_aval_actions(obs, list_of_action)
    
    if not move in available_action:
        #print(' bad prediction', available_action)
        bad_prediction = True
        
    obs = env.observe()[0].flatten()
    #print('move prediction', move)
    #env.render()
    #print('move', move)
    
    if not bad_prediction:
        #print('env step')
        grid, end, winner = env.step(move, print_grid=False)
        #env.render()
        
    action = move
    #print('###### action type #####',type(action))
    #env.render()
    
    if (not end) and (not bad_prediction):
        #optimal player play
        move = player_opt.act(grid)
        #print('optimal move', move)
        
        grid, end, winner = env.step(move, print_grid=False)
        #env.render()
    

    #transition set-off
    new_obs = env.observe()[0].flatten()
    if not bad_prediction:
        rew = env.reward(player='X')
    else:
        rew = -1
    transition = (obs, action, rew, end, new_obs)
    #print('transition', transition)
    replay_buffer.append(transition)
    
    
    if end or bad_prediction:
        #print('game finish')
        if bad_prediction:
            episode_bad_predict += 1
            
        bad_prediction = False
        #print('-------------------------------------------')
        #print('Game end, winner is player ' + str(winner))
        #print('Optimal player = ' +  Turns[0])
        #print('Random player = ' +  Turns[1])
        #env.render()
        env.reset()
        episode_reward += rew
        game += 1
        if game % 10 == 0:
            
            rew_buffer.append(episode_reward)
            bad_pred_buffer.append(episode_bad_predict)

            # Logging
            if game % 10 == 0:
                print()
                print('game', game)
                print('episode bad predi',episode_bad_predict)
                print('Avg bad predi', np.mean(bad_pred_buffer))
                print('episode rew',episode_reward)
                print('Avg Rew', np.mean(rew_buffer)) 
                print('Loss', loss)
            episode_reward = 0.0
            episode_bad_predict = 0.0
    
    
    # Start Gradient Step
    #print('replay_buffer', len(replay_buffer))
    transitions = random.sample(replay_buffer, BATCH_SIZE)
    
    obses = np.asarray([t[0] for t in transitions])
    #print('obsas', type(obses))
    actions = np.asarray([t[1] for t in transitions])
    #print('actions type', type(actions))
    #print(actions)
    rews = np.asarray([t[2] for t in transitions])
    dones = np.asarray([t[3] for t in transitions])
    new_obses = np.asarray([t[4] for t in transitions])
    
    obses_tensor = torch.as_tensor(obses, dtype=torch.float32)
    actions_tensor = torch.as_tensor(actions, dtype=torch.int64)#.unsqueeze(-1)
    rews_tensor = torch.as_tensor(rews, dtype=torch.float32).unsqueeze(-1)
    dones_tensor = torch.as_tensor(dones, dtype=torch.float32).unsqueeze(-1)
    #print('dones tensor', dones_tensor.shape)
    new_obses_tensor = torch.as_tensor(new_obses, dtype=torch.float32)
    
    # Compute Targets
    target_q_values = target_net(new_obses_tensor)
    #print('target_q_values',target_q_values.shape)
    mask_possible_action = torch.ne(new_obses_tensor,0)*-1
    #print('mask_possible_action',mask_possible_action.shape)
    possible_target_q_values = target_q_values * mask_possible_action
    #print('possible_target_q_values',possible_target_q_values.shape)
    max_target_q_values = possible_target_q_values.max(dim=1, keepdim=True)[0]
    #print('max_target_q_values', max_target_q_values.shape)
    
    targets = rews_tensor + GAMMA * (1 - dones_tensor) * max_target_q_values
    #print('targets', targets.shape)
    
    
    # Compute Loss
    
    q_values = policy_net(obses_tensor)
    q_values_max = q_values.max(dim=1, keepdim=True)[0]
    
    
    #print('q_values',q_values.shape,'q_values_max',q_values_max.shape)
    
    #action_q_values = torch.gather(input=q_values, dim=1, index=actions_tensor)
    
    #print('action_q_values',action_q_values.shape, 'targets',targets.shape)
    
    loss = nn.functional.smooth_l1_loss(q_values_max, targets)
    
    #print('loss', loss, loss.shape, type(loss))
#     max_q_idx = torch.argmax(possible_q_values, dim=1)[0]
#     max_q_idx_item = max_q_idx.detach().item()
    
    
    # Gradient Descent
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Update Target Network
    if step % TARGET_UPDATE_FREQ == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
    # Logging
    


|- - -|
|- - -|
|- - -|


game 10
episode bad predi 9.0
Avg bad predi 4.5
episode rew -10.0
Avg Rew -5.0
Loss tensor(0.3144, grad_fn=<SmoothL1LossBackward0>)

game 20
episode bad predi 8.0
Avg bad predi 5.666666666666667
episode rew -6.0
Avg Rew -5.333333333333333
Loss tensor(0.3901, grad_fn=<SmoothL1LossBackward0>)

game 30
episode bad predi 9.0
Avg bad predi 6.5
episode rew -10.0
Avg Rew -6.5
Loss tensor(0.3281, grad_fn=<SmoothL1LossBackward0>)

game 40
episode bad predi 8.0
Avg bad predi 6.8
episode rew -10.0
Avg Rew -7.2
Loss tensor(0.3219, grad_fn=<SmoothL1LossBackward0>)

game 50
episode bad predi 9.0
Avg bad predi 7.166666666666667
episode rew -10.0
Avg Rew -7.666666666666667
Loss tensor(0.3287, grad_fn=<SmoothL1LossBackward0>)

game 60
episode bad predi 9.0
Avg bad predi 7.428571428571429
episode rew -10.0
Avg Rew -8.0
Loss tensor(0.1881, grad_fn=<SmoothL1LossBackward0>)

game 70
episode bad predi 8.0
Avg bad predi 7.5
episode rew -8.0
Avg Rew -8.0
Loss tensor(0.2326, grad_fn=<

In [ ]:
obs_tensor = torch.as_tensor(obs, dtype=torch.float32)
print(obs_tensor.shape, obs_tensor)
output_target = target_net(obs_tensor.unsqueeze(0))


print(output_target)

In [19]:

def find_aval_actions(state_of_game,act):
  possible_actions = []
  for i in range(9):
    if state_of_game[act[i]] == 0:
      possible_actions.append(i)
  return possible_actions

In [25]:
obs = env.observe()[0]
print(obs)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [26]:
env.step(1)
env.step(3)

(array([[ 0.,  1.,  0.],
        [-1.,  0.,  0.],
        [ 0.,  0.,  0.]]),
 False,
 None)

In [27]:
obs = env.observe()[0]
available_action = find_aval_actions(obs, list_of_action)
print(available_action)

[0, 2, 4, 5, 6, 7, 8]
